In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import axes3d
pd.options.display.max_columns = None

Q3) Good start, job half done (relation between FSP,FSW,ACE,Result) Does good start imply positive match result?

In [8]:
# Aus Open = Hard Court
# French Open = Clay Court
# Wim = Grass Court
# US Open = Hard Court
df = pd.read_csv('FrenchOpen-men-2013.csv')
display(df)

,Player1,Player2,Round,Result,FNL.1,FNL.2,FSP.1,FSW.1,SSP.1,SSW.1,ACE.1,DBF.1,WNR.1,UFE.1,BPC.1,BPW.1,NPA.1,NPW.1,TPW.1,ST1.1,ST2.1,ST3.1,ST4.1,ST5.1,FSP.2,FSW.2,SSP.2,SSW.2,ACE.2,DBF.2,WNR.2,UFE.2,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,Pablo Carreno-Busta,Roger Federer,1,0,0,3,62,27,38,11,1,3.0,12,29,1,3,9,20,50,2,2,3.0,NaN,NaN,68,33,32,14,10,0.0,33,19,7,7,14,18,88,6,6,6.0,NaN,NaN
1,Somdev Devvarman,Daniel Munoz-De La Nava,1,1,3,0,62,54,38,22,7,3.0,26,20,5,8,12,21,120,6,6,7.0,NaN,NaN,52,35,48,24,0,2.0,40,47,1,16,22,25,106,3,3,5.0,NaN,NaN
2,Tobias Kamke,Paolo Lorenzi,1,1,3,2,62,53,38,15,4,6.0,42,55,10,22,14,32,140,6,6,3.0,0.0,6.0,46,42,54,31,6,8.0,39,54,10,18,19,27,139,3,3,6.0,6.0,3.0
3,Julien Benneteau,Ricardas Berankis,1,1,3,1,72,87,28,19,14,2.0,48,27,4,13,14,30,163,7,6,5.0,7.0,NaN,53,58,47,38,13,10.0,72,56,4,13,33,43,149,6,3,7.0,6.0,NaN
4,Lukas Lacko,Sam Querrey,1,0,0,3,52,31,48,22,4,4.0,21,24,1,1,3,5,72,3,4,4.0,NaN,NaN,58,39,42,19,10,1.0,42,37,4,7,12,13,93,6,6,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Rafael Nadal,Stanislas Wawrinka,5,1,3,0,75,40,25,11,5,0.0,31,17,7,12,13,16,96,6,6,6.0,NaN,NaN,60,30,40,12,3,4.0,23,40,1,5,16,30,64,2,3,1.0,NaN,NaN
121,Novak Djokovic,Tommy Haas,5,1,3,0,64,41,36,22,11,0.0,46,25,5,12,22,27,116,6,7,7.0,NaN,NaN,60,48,40,21,4,5.0,19,34,2,2,2,17,84,3,6,5.0,NaN,NaN
122,David Ferrer,Jo-Wilfried Tsonga,6,1,3,0,60,35,40,23,3,1.0,24,21,6,17,17,22,107,6,7,6.0,NaN,NaN,58,37,42,17,6,3.0,31,56,2,5,7,16,84,1,6,2.0,NaN,NaN
123,Novak Djokovic,Rafael Nadal,6,0,2,3,67,76,33,30,5,2.0,54,75,5,9,32,50,158,4,6,1.0,7.0,7.0,64,73,36,35,6,3.0,61,44,8,16,15,26,177,6,3,6.0,6.0,9.0


In [22]:
tournaments_removals = {'AusOpen-men-2013':['Player1','Player2'],
 'AusOpen-women-2013':['Player1','Player2','ST4.1','ST5.1','ST4.2','ST5.2'],
 'FrenchOpen-men-2013':['Player1','Player2'],
 'FrenchOpen-women-2013':['Player1','Player2','ST4.1','ST5.1','ST4.2','ST5.2'],
 'USOpen-men-2013':['Player1','Player2','WNR.1','UFE.1','WNR.2','UFE.2'],
 'USOpen-women-2013':['Player 1','Player 2','ST4.1','ST5.1','ST4.2','ST5.2','TPW.1','TPW.2'],
 'Wimbledon-men-2013':['Player1','Player2','TPW.1','TPW.2'],
 'Wimbledon-women-2013':['Player1','Player2','ST4.1','ST5.1','ST4.2','ST5.2','TPW.1','TPW.2'],
}

df1 = df.drop(tournaments_removals['FrenchOpen-men-2013'], axis=1)
corr_matrix = df1.corr()
# plt.figure(figsize=(25,25))
# sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)
# plt.show()


def give_weights(tournament,tournaments_removals,df):
    df1 = df.drop(tournaments_removals[tournament], axis=1)
    corr_matrix = df1.corr()
    corr_threshold = 0.5
    # important_features_pcorr = corr_matrix['Result'][corr_matrix['Result'] > corr_threshold].index.tolist()
    # important_features_ncorr = corr_matrix['Result'][corr_matrix['Result'] < -corr_threshold].index.tolist()
    # print("Important features Positive Correlation:", important_features_pcorr)
    # print("Important features Negative Correlation:", important_features_ncorr)
    return list(corr_matrix['Result'])

weights = give_weights('FrenchOpen-men-2013',tournaments_removals,df)
d = pd.DataFrame(weights[4:10] + weights[22:28],columns = ['Correlation Values'])
d.index = ['FSP.1','FSW.1','SSP.1','SSW.1','ACE.1','DBF.1','FSP.1','FSW.2','SSP.2','SSW.2','ACE.2','DBF.2']
display(d)

,Correlation Values
FSP.1,0.165377
FSW.1,0.140262
SSP.1,-0.165377
SSW.1,0.045168
ACE.1,0.237969
DBF.1,-0.142763
FSP.1,-0.082201
FSW.2,-0.196195
SSP.2,0.082201
SSW.2,-0.123538
